In [1]:
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [2]:
# Sample HTML content, replace this with the fetched HTML from your target URL
sample_content = '''
<html>
    <body>
        <div id="main">
            <div class="child">
                <input type="text" id="name" name="name">
            </div>
            <div class="child">
                <input type="email" id="email" name="email">
            </div>
            <div class="child">
                <input type="password" id="password" name="password">
            </div>
        </div>
        <div id="other">
            <input type="checkbox" id="subscribe" name="subscribe">
        </div>
    </body>
</html>
'''
sample_soup = BeautifulSoup(sample_content, 'html.parser')

In [3]:
ashby_url = "https://jobs.ashbyhq.com/eightsleep/70a2ba22-04be-45b0-823a-8fc348035e0f/application?utm_source=Simplify"
url = ashby_url

In [4]:
# async def main():
p = await async_playwright().start()
browser = await p.firefox.launch(headless=False)

In [5]:
page = await browser.new_page()

# Replace this with your actual URL and waiting logic
# await page.set_content(html_content)
# Navigate to the URL and wait for the content to load
await page.goto(url)

await page.wait_for_timeout(5000)

In [6]:
# Fetch the HTML content of the page
content = await page.content()

# Use BeautifulSoup to parse the HTML
soup = BeautifulSoup(content, 'html.parser')

In [7]:
# Find all input elements
inputs = soup.find_all('input')

# target_parent = find_application_container(inputs) # FUNC

In [8]:
len(inputs)

30

In [9]:
inputs[-1]

<input id="553c6f4d-54ae-4915-93dc-bf5f821e1375-labeled-radio-3" name="553c6f4d-54ae-4915-93dc-bf5f821e1375" type="radio"/>

In [10]:
'''Iteration 1'''
# # Identify the parent div containing most inputs
# # def find_application_container(inputs): # FUNC
# parent_candidates = {}
# for inp in inputs:
#     parent = inp.find_parent('div')
#     if parent:
#         # parent_id = parent.get('id', '')
#         # parent_cls = parent.get('class', '')
#         class_list = parent.get('class', '')
#         class_str = ' '.join(class_list)
        
#         # print('parent_id:', parent_id)
#         # parent_candidates[parent_id] = parent_candidates.get(parent_id, 0) + 1
#         print('parent_id:', class_str)
#         parent_candidates[class_str] = parent_candidates.get(class_str, 0) + 1

# # Assume the parent containing most inputs is the one we're interested in
# target_parent_id = max(parent_candidates, key=parent_candidates.get)
# print('target_parent_id:', target_parent_id)
# # target_parent = soup.find('div', {'id': target_parent_id})
# target_parent = soup.find('div', {'class': target_parent_id})

# # return target_parent # FUNC

'Iteration 1'

In [11]:
'''Iteration 2'''
def get_unique_path(element):
    path = []
    while element:
        if element.name:
            try:
                siblings = element.find_parent().find_all(element.name)
                index = siblings.index(element)
                path.append(f"{element.name}:{index}")
            except: pass
        element = element.find_parent() # break
    return ">".join(reversed(path))

def find_by_unique_path(soup, unique_path):
    tags = unique_path.split("/")
    element = soup

    for tag in tags:
        name, index = tag[:-1], int(tag[-1])
        element = element.find_all(name, recursive=False)[index]

    return element

def find_best_parent(element, parent_candidates, all_input_count):
    if element is None:
        return
    
    if element.name == 'div':
        class_list = element.get('class', [])
        # class_str = ' '.join(class_list)
        id = get_unique_path(element)
        
        # if class_str not in parent_candidates:
        if id not in parent_candidates:
            # parent_candidates[class_str] = {
            parent_candidates[id] = {
                'count': 0,
                'contains_submit': False
            }
        
        # parent_candidates[class_str]['count'] += 1
        parent_candidates[id]['count'] += 1
        
        # # Check if this div contains an input element that says "submit", "next", or "continue"
        # for submit_type in ["submit", "next", "continue"]:
        #     if element.find("input", {"type": submit_type}) or element.find("input", {"value": submit_type}):
        #         parent_candidates[class_str]['contains_submit'] = True
    
    # Recursive call to traverse up
    find_best_parent(element.find_parent('div'), parent_candidates, all_input_count)

def identify_best_parent(soup):
    inputs = soup.find_all('input')
    all_input_count = len(inputs)
    
    parent_candidates = {}
    
    for inp in inputs:
        find_best_parent(inp, parent_candidates, all_input_count)
    
    # # Filter candidates based on your conditions
    # best_candidate = None
    # for class_str, data in parent_candidates.items():
    #     if best_candidate is None:
    #         best_candidate = class_str
    #         continue
        
    #     # Condition 1: Contains submit, next, or continue
    #     if data['contains_submit']:
    #         best_candidate = class_str
    #         break
        
    #     # Condition 2: Contains over 90% of all input elements
    #     if data['count'] / all_input_count > 0.9:
    #         best_candidate = class_str
    #         break
        
    #     # Condition 3: Contains the most inputs (this will be the default if none of the above conditions are met)
    #     if data['count'] > parent_candidates[best_candidate]['count']:
    #         best_candidate = class_str

    print('parent_candidates:', parent_candidates)

    best_candidate = list(parent_candidates.keys())[0]
    for id, data in parent_candidates.items():
        if data['count'] > parent_candidates[best_candidate]['count']:
            best_candidate = id
    
    return best_candidate

In [12]:
# sample
target_parent = identify_best_parent(sample_soup)

parent_candidates: {'html:0>body:0>div:0>div:0': {'count': 1, 'contains_submit': False}, 'html:0>body:0>div:0': {'count': 3, 'contains_submit': False}, 'html:0>body:0>div:0>div:1': {'count': 1, 'contains_submit': False}, 'html:0>body:0>div:0>div:2': {'count': 1, 'contains_submit': False}, 'html:0>body:0>div:4': {'count': 1, 'contains_submit': False}}


In [13]:
target_parent

'html:0>body:0>div:0'

In [14]:
target_parent = identify_best_parent(soup)

parent_candidates: {'html:0>body:0>div:0>div:1>div:1>div:5>div:0>div:0>div:0': {'count': 1, 'contains_submit': False}, 'html:0>body:0>div:0>div:1>div:1>div:5>div:0>div:0': {'count': 1, 'contains_submit': False}, 'html:0>body:0>div:0>div:1>div:1>div:5>div:0': {'count': 30, 'contains_submit': False}, 'html:0>body:0>div:0>div:1>div:1>div:5': {'count': 30, 'contains_submit': False}, 'html:0>body:0>div:0>div:1>div:1': {'count': 30, 'contains_submit': False}, 'html:0>body:0>div:0>div:1': {'count': 30, 'contains_submit': False}, 'html:0>body:0>div:0': {'count': 30, 'contains_submit': False}, 'html:0>body:0>div:0>div:1>div:1>div:5>div:0>div:9>div:0>div:0': {'count': 1, 'contains_submit': False}, 'html:0>body:0>div:0>div:1>div:1>div:5>div:0>div:9>div:0': {'count': 29, 'contains_submit': False}, 'html:0>body:0>div:0>div:1>div:1>div:5>div:0>div:9': {'count': 29, 'contains_submit': False}, 'html:0>body:0>div:0>div:1>div:1>div:5>div:0>div:9>div:0>div:1': {'count': 1, 'contains_submit': False}, 'htm

In [15]:
target_parent

'html:0>body:0>div:0>div:1>div:1>div:5>div:0'

In [16]:
# target_element = await page.locator('xpath={}'.format(target_parent)).all()

In [ ]:
# found_element = find_by_unique_path(soup, target_parent)

In [44]:
'''Iteration 3'''
def find_parents(element, parent_candidates):
    if element is None:
        return
    
    if element.name == 'div':
        if id not in parent_candidates:
            parent_candidates.append(element)
        
    find_best_parent(element.find_parent('div'), parent_candidates) # traverse up via recursion


def identify_best_parent(soup):
    inputs = soup.find_all('input')
    # total = len(inputs)
    
    candidates = []
    for inp in inputs:
        find_parents(inp, candidates)
    
    # print('parent_candidates:', candidates)

    best = candidates[0]; highscore = 0
    for candidate in candidates:
        # if len(candidate.find_all('input')) / total > 0.9:
        contains = len(candidate.find_all('input'))
        if contains > highscore:
            best = candidate; highscore = contains
    
    return best

In [45]:
parent = identify_best_parent(soup)

In [46]:
print(parent)

<div aria-labelledby="job-application-form" id="form" role="tabpanel" tabindex="0"><div class="_autofillPane_z1j6s_461 ashby-application-form-autofill-pane _container_elimo_29" data-highlight="none"><div class="_root_1vgha_1 ashby-application-form-autofill-input-root" data-state="default"><input accept=".pdf,.doc,.docx,.odt,.rtf,application/pdf,application/msword,application/vnd.openxmlformats-officedocument.wordprocessingml.document,application/vnd.oasis.opendocument.text,application/rtf" autocomplete="off" style="display: none;" tabindex="-1" type="file"/><div class="_base_1vgha_31 ashby-application-form-autofill-input-base-layer"><div class="_content_1vgha_36"><div class="_text_1vgha_44"><div class="_iconContainer_1vgha_53"><svg class="_icon_1vgha_53 ashby-application-form-autofill-input-icon" fill="none" height="1em" viewbox="0 0 576 512"><path d="M234.7 42.7L197 56.8c-3 1.1-5 4-5 7.2s2 6.1 5 7.2l37.7 14.1L248.8 123c1.1 3 4 5 7.2 5s6.1-2 7.2-5l14.1-37.7L315 71.2c3-1.1 5-4 5-7.2s-2-

In [47]:
parent.find_all('input')

[<input accept=".pdf,.doc,.docx,.odt,.rtf,application/pdf,application/msword,application/vnd.openxmlformats-officedocument.wordprocessingml.document,application/vnd.oasis.opendocument.text,application/rtf" autocomplete="off" style="display: none;" tabindex="-1" type="file"/>,
 <input class="_input_1xsmr_28 _input_1dgff_33" id="_systemfield_name" name="_systemfield_name" placeholder="Type here..." required="" type="text" value=""/>,
 <input class="_input_1xsmr_28 _input_1dgff_33" id="_systemfield_email" name="_systemfield_email" placeholder="hello@example.com..." required="" type="email" value=""/>,
 <input accept=".pdf,.doc,.docx,.odt,.rtf,image/*,video/*,audio/*,application/pdf,application/msword,application/vnd.openxmlformats-officedocument.wordprocessingml.document,application/vnd.oasis.opendocument.text,application/rtf" autocomplete="off" id="_systemfield_resume" required="" style="display: none;" tabindex="-1" type="file"/>,
 <input class="_input_y2cw4_79" name="3306ccd6-4a4f-4f03

In [48]:
len(parent.find_all('input'))

30

In [63]:
# parent.find_all('div')
cs = []; gcs = []; ggcs = []
c_c = 0; c_gc = 0; c_ggc = 0
for c in iter(parent.children):
    print(c)
    cs.append(c); c_c += 1
    for gc in iter(c.children):
        print(gc)
        gcs.append(gc); c_gc = +1
        for ggc in iter(gc.children):
            print(ggc)
            ggcs.append(ggc); c_ggc += 1

<div class="_autofillPane_z1j6s_461 ashby-application-form-autofill-pane _container_elimo_29" data-highlight="none"><div class="_root_1vgha_1 ashby-application-form-autofill-input-root" data-state="default"><input accept=".pdf,.doc,.docx,.odt,.rtf,application/pdf,application/msword,application/vnd.openxmlformats-officedocument.wordprocessingml.document,application/vnd.oasis.opendocument.text,application/rtf" autocomplete="off" style="display: none;" tabindex="-1" type="file"/><div class="_base_1vgha_31 ashby-application-form-autofill-input-base-layer"><div class="_content_1vgha_36"><div class="_text_1vgha_44"><div class="_iconContainer_1vgha_53"><svg class="_icon_1vgha_53 ashby-application-form-autofill-input-icon" fill="none" height="1em" viewbox="0 0 576 512"><path d="M234.7 42.7L197 56.8c-3 1.1-5 4-5 7.2s2 6.1 5 7.2l37.7 14.1L248.8 123c1.1 3 4 5 7.2 5s6.1-2 7.2-5l14.1-37.7L315 71.2c3-1.1 5-4 5-7.2s-2-6.1-5-7.2L277.3 42.7 263.2 5c-1.1-3-4-5-7.2-5s-6.1 2-7.2 5L234.7 42.7zM461.4 48L496

In [64]:
print(c_c, c_gc, c_ggc)

3 1 25


In [66]:
ggcs

[<input accept=".pdf,.doc,.docx,.odt,.rtf,application/pdf,application/msword,application/vnd.openxmlformats-officedocument.wordprocessingml.document,application/vnd.oasis.opendocument.text,application/rtf" autocomplete="off" style="display: none;" tabindex="-1" type="file"/>,
 <div class="_base_1vgha_31 ashby-application-form-autofill-input-base-layer"><div class="_content_1vgha_36"><div class="_text_1vgha_44"><div class="_iconContainer_1vgha_53"><svg class="_icon_1vgha_53 ashby-application-form-autofill-input-icon" fill="none" height="1em" viewbox="0 0 576 512"><path d="M234.7 42.7L197 56.8c-3 1.1-5 4-5 7.2s2 6.1 5 7.2l37.7 14.1L248.8 123c1.1 3 4 5 7.2 5s6.1-2 7.2-5l14.1-37.7L315 71.2c3-1.1 5-4 5-7.2s-2-6.1-5-7.2L277.3 42.7 263.2 5c-1.1-3-4-5-7.2-5s-6.1 2-7.2 5L234.7 42.7zM461.4 48L496 82.6 386.2 192.3l-34.6-34.6L461.4 48zM80 429.4L317.7 191.7l34.6 34.6L114.6 464 80 429.4zM427.4 14.1L46.1 395.4c-18.7 18.7-18.7 49.1 0 67.9l34.6 34.6c18.7 18.7 49.1 18.7 67.9 0L529.9 116.5c18.7-18.7 18.7

In [71]:
for ggc in ggcs:
    print(ggc.text)


Autofill from resumeUpload your resume here to autofill key application fields.Upload file
Drop your resume here!
Parsing your resume. Autofilling key fields...
Name
Email
ResumeUpload Fileor drag and drop here
Why do you want to join Eight Sleep?
Are you legally authorized to work in the United States?YesNo
What degree are you currently pursuing?Undergraduate degreeMasters degreeOther
Are you currently located in Boston area or willing to relocate on your own?YesNo
Are you willing to work onsite at our Boston office (located in Newton Centre) at least 3 days a week?YesNo
Are you able to work a normal 9-5 schedule for 40 hours per week, with occasional weekend work (monitoring study data for a couple hours on the weekend)?YesNo
Are you available from January 2nd - June 30th 2024 for 40 hours per week?YesNo
Are you aware this position is a Data Engineering co-op, not Machine Learning? The main tasks will include: 1. Manipulating, cleaning, and transforming raw data into a usable format

Shit ain't that simple. Need some sort of holisticism—a heuristic—in traversing into just the right level; the right depth. Using ChatGPT w/ templating as a question-unit (vs. redundant / multi-question)  classifier seems like the best approach. Also need to classify into short answer, file upload, open-ended (multiline), yes/no, selector, and tags.

Also... I think the right way to do this, in order to not lose out on crucial information in the case of complex forms where prompts are introduced at 2-3 levels of nesting, is:
- go through all the children/grandchildren
- recursively remove information that are present within grandgrandchildren
- so, upper levels will have information. but only unique information. which means, information redundancy or leaking will be avoided to an extent. 
- reckon we can hardcode nesting depths w/ reasonable effectiveness. this need to be validated w/ testing. in downstream LLM-augmented classification stage, we can ask if it thinks the question (or some parts of the question) is accidentally omitted. in this case, we can go up the nesting levels until it is satisfied with the information. thanks to hierarchical deduplication earlier, information from totally unrelated questions/forms will not leak into the instantaneous question-unit (QU). if anything, it will only contain tangentially related stuff (since all the sibling-level contents have been removed already!) in addition, if it thinks that it accidentally ended up with multiple questions, we can help it go down a level to answer.

쒸바 테스트를 해봐야 이게 충분한가, 안 충분한가 한번 재어볼 수가 있는데, 테스트 없이 블랙박스로 짜니까 계속 고민만 하게 되고 아무런 성과가 없네. 일단 최대한 간단한 테스트 케이스부터 작성해놓고 어떻게 더 진행할까, 말까 마지막으로 결정해보자. 

---

### Questions for ChatGPT

Thank you! This is my current algorithm that you suggested. I would like it to traverse up not only one but multiple levels, until any of three conditions is met:
- the target parent contains an input element that says "submit", "next", or "continue"
- the target parent contains over 90% of all input elements in the page
- the target parent contains the most inputs (compared to any other parent candidates)

Thank you ! Here is the algorithm to find the main application div ("target parent")  according to the solution you suggested. There is a problem, though: the elements do not 

In [ ]:

# Loop through the immediate children and fill in the inputs
for child in target_parent.find_all('div', recursive=False):
    input_elem = child.find('input')
    if input_elem:
        input_name = input_elem.get('name', '')
        input_type = input_elem.get('type', '')
        
        # Use Python's input function to get user input
        user_input = input(f"Enter your {input_name} ({input_type}): ")
        
        # Fill in the input field using Playwright
        await page.fill(f'input[name="{input_name}"]', user_input)

# Do something after filling all the inputs, e.g., submit the form
# await page.click("#submit-button")

# await browser.close()